# core

> core functions

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export 
def interpolation_search(o, target, low=None, high=None, how='left'):
    """Interpolation search algorithm to find the closest value to `target` in the sorted list `o`."""

    assert how in ['left', 'right']

    if low is None:
        low = 0
    if high is None:
        high = len(o)
    
    assert o[low] <= target <= o[high]

    pos = low + ((int(target) - o[low]) * (high - low)) // (o[high] - o[low])

    if o[pos] < target:
        if o[pos + 1] > target:
            if how=='left':
                return pos
            else:
                return pos + 1
        return interpolation_search(o, target, pos + 1, high, how=how)
    else:
        if o[pos - 1] < target:
            if how=='right':
                return pos
            else:
                return pos - 1
        return interpolation_search(o, target, low, pos - 1, how=how)

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()